In [1]:
import requests
import json
url = "http://papers.cumincad.org/cgi-bin/works/Search?search=&x=47&y=8&max=1000"

In [2]:
def main(url, fileName):
    """Run the other functions to turn a CumInCAD search into a json of published paper data"""
    
    # Important initialisations
    nextURL = url
    listings = []
    pageCount = 0
    
    # Loop through each page of search results starting at the specified url, gathering links to listings
    while nextURL != "lastPage":
        # Get the HTML
        html = getHTML(nextURL)
        # Split the HTML into listings and add to the list of listings
        listings.extend(getListings(html))
        # Find the link to the next page
        nextURL = getNextURL(html)
        # Count and display the amount of pages
        pageCount += 1
        print("Page number " + str(pageCount))
    
    # Extract just the URL of each listing
    listings = [getListingURL(listing) for listing in listings]
    
    # Looping through each of the listing pages to mine data
    listingCount = 0
    listingsData = []
    for listing in listings:
        # Counting the listings
        listingCount += 1
        print("Listing " + str(listingCount))
        
        # Getting the HTML
        listingHTML = getHTML(listing)
        
        # Scraping the metadata from the html
        dataPoints = scrapeListingMeta(listingHTML)
        
        # Adding the data to the large collection of it
        listingsData.append(dataPoints)
    
    # Save the data as a json string
    jsonDataString = json.dumps(listingsData)
    
    # Save the string to a file
    with open(fileName + ".json", 'w') as jsonFile:
        jsonFile.write(jsonDataString)

In [3]:
def getNextURL(html):
    """Find the next page's URL from within the previous's HTML"""
    
    # Split the html at the point where the next page icon is used
    parts = html.split('="/woda/icons/flat-noborder/forward.gif"')
    
    # If this icon doesn't occur, return the 'lastPage' flag
    if len(parts) == 1:
        return "lastPage"
    else:
        # Getting the URL
        nextURL = "http://papers.cumincad.org" + parts[0].split("HREF")[-1].split('"')[1]
        
        # Replacing &amp; with &
        nextURL = nextURL.replace("&amp;", "&")
        
        # Returning the new URL
        return nextURL

In [4]:
def getHTML(url):
    """Get the HTML at a given URL"""
    response = requests.get(url)
    return response.text

In [5]:
def getListings(fullHTML):
    """Get the listings from the HTML"""
    
    # Cut down to the relevant <div> 
    div = fullHTML.split("<DIV CLASS=RECORDS>")[1].split("</DIV>")[0]
    
    # Get all the table rows, excluding the bit of non-table stuff at the beginning and the first header row
    table = [("<tr" + row).strip() for row in div.split("<tr")[2:]]
    
    # Get rid of a little extraneous bit at the end of the last row
    if table[-1].endswith("</tbody></table><div><br/></div>"):
        table[-1] = table[-1][:-32].strip()
    
    # Joining rows that shouldn't be seperated (there's one row inside each row)
    rowPairs = list(zip(table[::2], table[1::2]))
    rows = [pair[0] + pair[1] for pair in rowPairs]
    return rows

In [6]:
def getListingURL(listing):
    """Get the URL that corresponds to the detailed page of a listing"""
    return listing.split("<A HREF=")[1].split(">")[0]

In [7]:
def scrapeListingMeta(html):
    """Return a dictionary of all the metadata in the html of a listing's page"""
    
    # Retrieving a list of each meta point in the HTML
    metaPoints = html.split("<meta")[1:]
    metaPoints = [point.split("/>")[0] for point in metaPoints]
    
    # Looping through the points, adding the data from each string to a dictionary
    listingMeta = {}
    badListingMeta = {}
    for point in metaPoints:
        # Split the point up into valuable information
        point = point.strip()
        pointParts = point.split('"')
        
        # Add the parts appropriately to the dictionary
        listingMeta[pointParts[1]] = pointParts[3]
        
        # Note down the potentially bad data in a different dictionary
        if point.count('"') != 4:
            badListingMeta[pointParts[1]] = pointParts[3]
    
    
    #print("\n\nlistingMeta:")
    #[print(meta + ": " + listingMeta[meta]) for meta in listingMeta]
    #print("\nbadListingMeta:")
    #[print(meta + ": " + badListingMeta[meta]) for meta in badListingMeta]
    
    return listingMeta

In [ ]:
main(url, "allSearchData")

Page number 1
Page number 2
Page number 3
Page number 4
Page number 5
Page number 6
Page number 7
Page number 8
Page number 9
Page number 10
Page number 11
Page number 12
Page number 13
Page number 14
Page number 15
Listing 1
Listing 2
Listing 3
Listing 4
Listing 5
Listing 6
Listing 7
Listing 8
Listing 9
Listing 10
Listing 11
Listing 12
Listing 13
Listing 14
Listing 15
Listing 16
Listing 17
Listing 18
Listing 19
Listing 20
Listing 21
Listing 22
Listing 23
Listing 24
Listing 25
Listing 26
Listing 27
Listing 28
Listing 29
Listing 30
Listing 31
Listing 32
Listing 33
Listing 34
Listing 35
Listing 36
Listing 37
Listing 38
Listing 39
Listing 40
Listing 41
Listing 42
Listing 43
Listing 44
Listing 45
Listing 46
Listing 47
Listing 48
Listing 49
Listing 50
Listing 51
Listing 52
Listing 53
Listing 54
Listing 55
Listing 56
Listing 57
Listing 58
Listing 59
Listing 60
Listing 61
Listing 62
Listing 63
Listing 64
Listing 65
Listing 66
Listing 67
Listing 68
Listing 69
Listing 70
Listing 71
Listing 72
L

Listing 675
Listing 676
Listing 677
Listing 678
Listing 679
Listing 680
Listing 681
Listing 682
Listing 683
Listing 684
Listing 685
Listing 686
Listing 687
Listing 688
Listing 689
Listing 690
Listing 691
Listing 692
Listing 693
Listing 694
Listing 695
Listing 696
Listing 697
Listing 698
Listing 699
Listing 700
Listing 701
Listing 702
Listing 703
Listing 704
Listing 705
Listing 706
Listing 707
Listing 708
Listing 709
Listing 710
Listing 711
Listing 712
Listing 713
Listing 714
Listing 715
Listing 716
Listing 717
Listing 718
Listing 719
Listing 720
Listing 721
Listing 722
Listing 723
Listing 724
Listing 725
Listing 726
Listing 727
Listing 728
Listing 729
Listing 730
Listing 731
Listing 732
Listing 733
Listing 734
Listing 735
Listing 736
Listing 737
Listing 738
Listing 739
Listing 740
Listing 741
Listing 742
Listing 743
Listing 744
Listing 745
Listing 746
Listing 747
Listing 748
Listing 749
Listing 750
Listing 751
Listing 752
Listing 753
Listing 754
Listing 755
Listing 756
Listing 757
List

Listing 1331
Listing 1332
Listing 1333
Listing 1334
Listing 1335
Listing 1336
Listing 1337
Listing 1338
Listing 1339
Listing 1340
Listing 1341
Listing 1342
Listing 1343
Listing 1344
Listing 1345
Listing 1346
Listing 1347
Listing 1348
Listing 1349
Listing 1350
Listing 1351
Listing 1352
Listing 1353
Listing 1354
Listing 1355
Listing 1356
Listing 1357
Listing 1358
Listing 1359
Listing 1360
Listing 1361
Listing 1362
Listing 1363
Listing 1364
Listing 1365
Listing 1366
Listing 1367
Listing 1368
Listing 1369
Listing 1370
Listing 1371
Listing 1372
Listing 1373
Listing 1374
Listing 1375
Listing 1376
Listing 1377
Listing 1378
Listing 1379
Listing 1380
Listing 1381
Listing 1382
Listing 1383
Listing 1384
Listing 1385
Listing 1386
Listing 1387
Listing 1388
Listing 1389
Listing 1390
Listing 1391
Listing 1392
Listing 1393
Listing 1394
Listing 1395
Listing 1396
Listing 1397
Listing 1398
Listing 1399
Listing 1400
Listing 1401
Listing 1402
Listing 1403
Listing 1404
Listing 1405
Listing 1406
Listing 1407

Listing 1962
Listing 1963
Listing 1964
Listing 1965
Listing 1966
Listing 1967
Listing 1968
Listing 1969
Listing 1970
Listing 1971
Listing 1972
Listing 1973
Listing 1974
Listing 1975
Listing 1976
Listing 1977
Listing 1978
Listing 1979
Listing 1980
Listing 1981
Listing 1982
Listing 1983
Listing 1984
Listing 1985
Listing 1986
Listing 1987
Listing 1988
Listing 1989
Listing 1990
Listing 1991
Listing 1992
Listing 1993
Listing 1994
Listing 1995
Listing 1996
Listing 1997
Listing 1998
Listing 1999
Listing 2000
Listing 2001
Listing 2002
Listing 2003
Listing 2004
Listing 2005
Listing 2006
Listing 2007
Listing 2008
Listing 2009
Listing 2010
Listing 2011
Listing 2012
Listing 2013
Listing 2014
Listing 2015
Listing 2016
Listing 2017
Listing 2018
Listing 2019
Listing 2020
Listing 2021
Listing 2022
Listing 2023
Listing 2024
Listing 2025
Listing 2026
Listing 2027
Listing 2028
Listing 2029
Listing 2030
Listing 2031
Listing 2032
Listing 2033
Listing 2034
Listing 2035
Listing 2036
Listing 2037
Listing 2038

Listing 2593
Listing 2594
Listing 2595
Listing 2596
Listing 2597
Listing 2598
Listing 2599
Listing 2600
Listing 2601
Listing 2602
Listing 2603
Listing 2604
Listing 2605
Listing 2606
Listing 2607
Listing 2608
Listing 2609
Listing 2610
Listing 2611
Listing 2612
Listing 2613
Listing 2614
Listing 2615
Listing 2616
Listing 2617
Listing 2618
Listing 2619
Listing 2620
Listing 2621
Listing 2622
Listing 2623
Listing 2624
Listing 2625
Listing 2626
Listing 2627
Listing 2628
Listing 2629
Listing 2630
Listing 2631
Listing 2632
Listing 2633
Listing 2634
Listing 2635
Listing 2636
Listing 2637
Listing 2638
Listing 2639
Listing 2640
Listing 2641
Listing 2642
Listing 2643
Listing 2644
Listing 2645
Listing 2646
Listing 2647
Listing 2648
Listing 2649
Listing 2650
Listing 2651
Listing 2652
Listing 2653
Listing 2654
Listing 2655
Listing 2656
Listing 2657
Listing 2658
Listing 2659
Listing 2660
Listing 2661
Listing 2662
Listing 2663
Listing 2664
Listing 2665
Listing 2666
Listing 2667
Listing 2668
Listing 2669

...

Listing 10739
Listing 10740
Listing 10741
Listing 10742
Listing 10743
Listing 10744
Listing 10745
Listing 10746
Listing 10747
Listing 10748
Listing 10749
Listing 10750
Listing 10751
Listing 10752
Listing 10753
Listing 10754
Listing 10755
Listing 10756
Listing 10757
Listing 10758
Listing 10759
Listing 10760
Listing 10761
Listing 10762
Listing 10763
Listing 10764
Listing 10765
Listing 10766
Listing 10767
Listing 10768
Listing 10769
Listing 10770
Listing 10771
Listing 10772
Listing 10773
Listing 10774
Listing 10775
Listing 10776
Listing 10777
Listing 10778
Listing 10779
Listing 10780
Listing 10781
Listing 10782
Listing 10783
Listing 10784
Listing 10785
Listing 10786
Listing 10787
Listing 10788
Listing 10789
Listing 10790
Listing 10791
Listing 10792
Listing 10793
Listing 10794
Listing 10795
Listing 10796
Listing 10797
Listing 10798
Listing 10799
Listing 10800
Listing 10801
Listing 10802
Listing 10803
Listing 10804
Listing 10805
Listing 10806
Listing 10807
Listing 10808
Listing 10809
Listin

Listing 11325
Listing 11326
Listing 11327
Listing 11328
Listing 11329
Listing 11330
Listing 11331
Listing 11332
Listing 11333
Listing 11334
Listing 11335
Listing 11336
Listing 11337
Listing 11338
Listing 11339
Listing 11340
Listing 11341
Listing 11342
Listing 11343
Listing 11344
Listing 11345
Listing 11346
Listing 11347
Listing 11348
Listing 11349
Listing 11350
Listing 11351
Listing 11352
Listing 11353
Listing 11354
Listing 11355
Listing 11356
Listing 11357
Listing 11358
Listing 11359
Listing 11360
Listing 11361
Listing 11362
Listing 11363
Listing 11364
Listing 11365
Listing 11366
Listing 11367
Listing 11368
Listing 11369
Listing 11370
Listing 11371
Listing 11372
Listing 11373
Listing 11374
Listing 11375
Listing 11376
Listing 11377
Listing 11378
Listing 11379
Listing 11380
Listing 11381
Listing 11382
Listing 11383
Listing 11384
Listing 11385
Listing 11386
Listing 11387
Listing 11388
Listing 11389
Listing 11390
Listing 11391
Listing 11392
Listing 11393
Listing 11394
Listing 11395
Listin

Listing 11911
Listing 11912
Listing 11913
Listing 11914
Listing 11915
Listing 11916
Listing 11917
Listing 11918
Listing 11919
Listing 11920
Listing 11921
Listing 11922
Listing 11923
Listing 11924
Listing 11925
Listing 11926
Listing 11927
Listing 11928
Listing 11929
Listing 11930
Listing 11931
Listing 11932
Listing 11933
Listing 11934
Listing 11935
Listing 11936
Listing 11937
Listing 11938
Listing 11939
Listing 11940
Listing 11941
Listing 11942
Listing 11943
Listing 11944
Listing 11945
Listing 11946
Listing 11947
Listing 11948
Listing 11949
Listing 11950
Listing 11951
Listing 11952
Listing 11953
Listing 11954
Listing 11955
Listing 11956
Listing 11957
Listing 11958
Listing 11959
Listing 11960
Listing 11961
Listing 11962
Listing 11963
Listing 11964
Listing 11965
Listing 11966
Listing 11967
Listing 11968
Listing 11969
Listing 11970
Listing 11971
Listing 11972
Listing 11973
Listing 11974
Listing 11975
Listing 11976
Listing 11977
Listing 11978
Listing 11979
Listing 11980
Listing 11981
Listin

Listing 12497
Listing 12498
Listing 12499
Listing 12500
Listing 12501
Listing 12502
Listing 12503
Listing 12504
Listing 12505
Listing 12506
Listing 12507
Listing 12508
Listing 12509
Listing 12510
Listing 12511
Listing 12512
Listing 12513
Listing 12514
Listing 12515
Listing 12516
Listing 12517
Listing 12518
Listing 12519
Listing 12520
Listing 12521
Listing 12522
Listing 12523
Listing 12524
Listing 12525
Listing 12526
Listing 12527
Listing 12528
Listing 12529
Listing 12530
Listing 12531
Listing 12532
Listing 12533
Listing 12534
Listing 12535
Listing 12536
Listing 12537
Listing 12538
Listing 12539
Listing 12540
Listing 12541
Listing 12542
Listing 12543
Listing 12544
Listing 12545
Listing 12546
Listing 12547
Listing 12548
Listing 12549
Listing 12550
Listing 12551
Listing 12552
Listing 12553
Listing 12554
Listing 12555
Listing 12556
Listing 12557
Listing 12558
Listing 12559
Listing 12560
Listing 12561
Listing 12562
Listing 12563
Listing 12564
Listing 12565
Listing 12566
Listing 12567
Listin

Listing 13083
Listing 13084
Listing 13085
Listing 13086
Listing 13087
Listing 13088
Listing 13089
Listing 13090
Listing 13091
Listing 13092
Listing 13093
Listing 13094
Listing 13095
Listing 13096
Listing 13097
Listing 13098
Listing 13099
Listing 13100
Listing 13101
Listing 13102
Listing 13103
Listing 13104
Listing 13105
Listing 13106
Listing 13107
Listing 13108
Listing 13109
Listing 13110
Listing 13111
Listing 13112
Listing 13113
Listing 13114
Listing 13115
Listing 13116
Listing 13117
Listing 13118
Listing 13119
Listing 13120
Listing 13121
Listing 13122
Listing 13123
Listing 13124
Listing 13125
Listing 13126
Listing 13127
Listing 13128
Listing 13129
Listing 13130
Listing 13131
Listing 13132
Listing 13133
Listing 13134
Listing 13135
Listing 13136
Listing 13137
Listing 13138
Listing 13139
Listing 13140
Listing 13141
Listing 13142
Listing 13143
Listing 13144
Listing 13145
Listing 13146
Listing 13147
Listing 13148
Listing 13149
Listing 13150
Listing 13151
Listing 13152
Listing 13153
Listin